In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import gc

In [11]:
# Reset Notebook (If Overloaded)
%reset -f

In [14]:
# Clear Garbage if memory overloaded
gc.collect()

484

In [9]:
# Set dataset directory
d_dir = '../../datasets'

In [10]:
# Read data file
input_file = f'{d_dir}/univ2_packet_trace.csv'
packet_trace = pd.read_csv(input_file)

In [11]:
packet_trace.head()

,timestamp,src_ip,dst_ip,src_port,dst_port,protocol,pkt_size
0,1264186933,244.157.46.108,244.157.82.156,7001,7000,17,86
1,1264186933,244.157.82.156,244.157.46.108,7000,7001,17,108
2,1264186933,244.157.46.108,244.157.82.156,7001,7000,17,107
3,1264186933,244.157.82.156,244.157.46.108,7000,7001,17,190
4,1264186933,244.157.46.108,244.157.82.156,7001,7000,17,107


In [12]:
packet_trace.describe()

,timestamp,src_port,dst_port,protocol,pkt_size
count,6.979085e+06,6.979085e+06,6.979085e+06,6.979085e+06,6.979085e+06
mean,1.264187e+09,7.105508e+03,7.070055e+03,1.701792e+01,7.518363e+02
std,1.939861e+02,2.419455e+03,2.098364e+03,1.475359e+00,6.803934e+02
min,1.264187e+09,1.700000e+01,2.100000e+01,1.000000e+00,6.000000e+01
25%,1.264187e+09,7.000000e+03,7.000000e+03,1.700000e+01,1.070000e+02
50%,1.264187e+09,7.001000e+03,7.000000e+03,1.700000e+01,1.900000e+02
75%,1.264187e+09,7.001000e+03,7.001000e+03,1.700000e+01,1.486000e+03
max,1.264188e+09,6.553200e+04,6.553200e+04,1.030000e+02,1.514000e+03


In [13]:
packet_trace.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6979085 entries, 0 to 6979084
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  int64 
 1   src_ip     object
 2   dst_ip     object
 3   src_port   int64 
 4   dst_port   int64 
 5   protocol   int64 
 6   pkt_size   int64 
dtypes: int64(5), object(2)
memory usage: 372.7+ MB


In [14]:
def get_duration(start_time, end_time):
    timestamp1 = datetime.fromtimestamp(start_time)
    timestamp2 = datetime.fromtimestamp(end_time)

    # Calculate the duration
    duration = timestamp2 - timestamp1

    if duration.seconds >= 60:
        duration = f'{int(duration.seconds / 60)} mins'
    else:
        duration = f'{int(duration.seconds)} seconds'
    
    return duration

In [15]:
period = get_duration(packet_trace['timestamp'].iloc[0], packet_trace['timestamp'].iloc[-1])

unique_src_ips = len(packet_trace['src_ip'].unique())
unique_dst_ips = len(packet_trace['dst_ip'].unique())

unique_src_ports = len(packet_trace['src_port'].unique())
unique_dst_ports = len(packet_trace['dst_port'].unique())

unique_protocols = len(packet_trace['protocol'].unique())

print(f'Total Time Period: {period}')
print(f'Unique Source IPs: {unique_src_ips}, Unique Destination IPs: {unique_dst_ips}')
print(f'Unique Source Ports: {unique_src_ports}, Unique Destination Ports: {unique_dst_ports}')
print(f'Unique Protocols: {unique_protocols}')

Total Time Period: 11 mins
Unique Source IPs: 1668, Unique Destination IPs: 1599
Unique Source Ports: 7508, Unique Destination Ports: 7059
Unique Protocols: 6


In [16]:
packet_trace.isnull().sum()

timestamp    0
src_ip       0
dst_ip       0
src_port     0
dst_port     0
protocol     0
pkt_size     0
dtype: int64

In [21]:
def get_flows(data, min_timeout = 1, max_timeout = None):
    # Create a dictionary to store flow information
    flows = {}

    # Iterate through the dataset
    for index, row in data.iterrows():
        # Extract relevant packet attributes 
        timestamp = row['timestamp']
        src_ip = row['src_ip']
        dst_ip = row['dst_ip']
        protocol = row['protocol']
        src_port = row['src_port']
        dst_port = row['dst_port']
        pkt_size = row['pkt_size']

        if protocol == 6 or protocol == 17:
            # Create a unique key for the flow based on the packet attributes
            flow_key = (src_ip, dst_ip, src_port, dst_port, protocol)

            # Check if the flow already exists in the dictionary
            if flow_key in flows:
                flows[flow_key]['packet_count'] += 1

                # maximum inter arrival time
                if flows[flow_key]['max_iat'] < row['timestamp'] - flows[flow_key]['end_time']:
                    flows[flow_key]['max_iat'] = row['timestamp'] - flows[flow_key]['end_time']

                # idle timeout
                if flows[flow_key]['max_iat'] < min_timeout:
                    flows[flow_key]['idle_timeout'] = min_timeout

                elif flows[flow_key]['max_iat'] > max_timeout:
                    flows[flow_key]['idle_timeout'] = max_timeout

                else:
                    flows[flow_key]['idle_timeout'] = flows[flow_key]['max_iat']

                # flow duration
                flows[flow_key]['flow_duration'] = row['timestamp'] - flows[flow_key]['start_time']
                
                flows[flow_key]['end_time'] = row['timestamp']
                
            else:
                # Create a new entry for the flow
                flows[flow_key] = {
                    'flow_key': flow_key,
                    'start_time': timestamp,
                    'end_time': timestamp,
                    'source_ip': src_ip,
                    'destination_ip': dst_ip,
                    'protocol': protocol,
                    'source_port': src_port,
                    'destination_port': dst_port,
                    'first_pkt_size': pkt_size,
                    'max_iat': 0,
                    'last_packet_time': 0,
                    'flow_duration': 0,
                    'packet_count': 1,
                    'idle_timeout': 0,
                }
            
        if index % 100000 == 0:
            print(f'{index} rows processed')
        
    # Output csv file
    # Convert dictionary to dataframe
    data_features = pd.DataFrame.from_dict(flows)

    # Transpose the dataframe
    data_features = data_features.transpose()

    return data_features

In [22]:
# Extract flows
min_timeout = 1
max_timeout = 11

flows = get_flows(packet_trace, min_timeout, max_timeout)

0 rows processed
100000 rows processed
200000 rows processed
300000 rows processed
400000 rows processed
500000 rows processed
600000 rows processed
700000 rows processed
800000 rows processed
900000 rows processed
1000000 rows processed
1100000 rows processed
1200000 rows processed
1300000 rows processed
1400000 rows processed
1500000 rows processed
1600000 rows processed
1700000 rows processed
1800000 rows processed
1900000 rows processed
2000000 rows processed
2100000 rows processed
2200000 rows processed
2300000 rows processed
2400000 rows processed
2500000 rows processed
2600000 rows processed
2700000 rows processed
2800000 rows processed
2900000 rows processed
3000000 rows processed
3100000 rows processed
3200000 rows processed
3300000 rows processed
3400000 rows processed
3500000 rows processed
3600000 rows processed
3700000 rows processed
3800000 rows processed
3900000 rows processed
4000000 rows processed
4100000 rows processed
4200000 rows processed
4300000 rows processed
440

In [24]:
# Remove timestamp column
flows = flows.drop('flow_key', axis=1)

In [25]:
flows = flows.drop('last_packet_time', axis=1)

In [26]:
def get_flow_class(flow_duration, pkt_count):
    if flow_duration > 11 and pkt_count > 10:
        flow_class = 3
    elif flow_duration <= 2 and pkt_count <= 2:
        flow_class = 1
    else:
        flow_class = 2

    return flow_class

In [28]:
# Adding Flow Class
flows['flow_class'] = np.vectorize(get_flow_class)(flows['flow_duration'], flows['packet_count'])
flows

,,,,,start_time,end_time,source_ip,destination_ip,protocol,source_port,destination_port,first_pkt_size,max_iat,flow_duration,packet_count,idle_timeout,flow_class
244.157.46.108,244.157.82.156,7001,7000,17,1264186933,1264187420,244.157.46.108,244.157.82.156,17,7001,7000,86,193,487,121,11,3
244.157.82.156,244.157.46.108,7000,7001,17,1264186933,1264187420,244.157.82.156,244.157.46.108,17,7000,7001,108,193,487,111,11,3
244.157.193.38,244.157.82.13,7001,7000,17,1264186933,1264187486,244.157.193.38,244.157.82.13,17,7001,7000,86,309,553,38220,11,3
244.157.82.13,244.157.193.38,7000,7001,17,1264186933,1264187486,244.157.82.13,244.157.193.38,17,7000,7001,190,309,553,25112,11,3
244.157.48.193,244.157.82.167,7001,7000,17,1264186933,1264187614,244.157.48.193,244.157.82.167,17,7001,7000,86,115,681,423,11,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244.157.117.241,244.157.82.23,52980,7002,17,1264187634,1264187634,244.157.117.241,244.157.82.23,17,52980,7002,107,0,0,1,0,1
244.157.61.145,244.157.82.23,7001,7003,17,1264187634,1264187634,244.157.61.145,244.157.82.23,17,7001,7003,86,0,0,6,1,2
244.157.82.23,244.157.61.145,7003,7001,17,1264187634,1264187634,244.157.82.23,244.157.61.145,17,7003,7001,1118,0,0,3,1,2
244.157.193.84,244.157.82.43,56059,7002,17,1264187635,1264187635,244.157.193.84,244.157.82.43,17,56059,7002,334,0,0,14,1,2


In [29]:
output_file = f'{d_dir}/univ2_flows.csv'

# export data to csv file
flows.to_csv(output_file, index=False)

print('Data Expoted.')

Data Expoted.
